In [18]:
!pip install langchain chromadb transformers google-cloud-aiplatform google-auth google-search-results > /dev/null

In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/content/generativeaitrial-trialLC.json'

In [2]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import VertexAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [3]:
from langchain.chains.summarize import load_summarize_chain

llm = VertexAI(temperature=0)
summary_chain = load_summarize_chain(llm, 
                                     chain_type="map_reduce")

In [4]:
from langchain.chains import AnalyzeDocumentChain

summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)

In [5]:
!unzip lc_documentdb.zip

Archive:  lc_documentdb.zip
   creating: content/lc_documentdb/
  inflating: content/lc_documentdb/chroma-embeddings.parquet  
   creating: content/lc_documentdb/index/
  inflating: content/lc_documentdb/index/uuid_to_id_b973adbd-91f7-4c95-b071-c5aabdbd5ab7.pkl  
  inflating: content/lc_documentdb/index/id_to_uuid_b973adbd-91f7-4c95-b071-c5aabdbd5ab7.pkl  
  inflating: content/lc_documentdb/index/index_b973adbd-91f7-4c95-b071-c5aabdbd5ab7.bin  
  inflating: content/lc_documentdb/index/index_metadata_b973adbd-91f7-4c95-b071-c5aabdbd5ab7.pkl  
  inflating: content/lc_documentdb/chroma-collections.parquet  


In [6]:
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

embeddings = VertexAIEmbeddings()

persist_directory = '/content/content/lc_documentdb'

vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embeddings)

db_retriever = vectordb.as_retriever()


langchain_qa = RetrievalQA.from_chain_type(llm=VertexAI(), 
                                 chain_type="stuff", 
                                 retriever=db_retriever)

In [16]:
def summarize_output(topic):
  relevant_data = langchain_qa(topic)
  #print(relevant_data)
  summary = summarize_document_chain.run(relevant_data['result'])
  return summary

In [8]:
query = 'List the langchain Agents'

In [33]:
summarize_output(query)

'Action agents act without planning, while plan-and-execute agents plan before acting.'

In [24]:
from langchain.chains.question_answering import load_qa_chain

qa_chain = load_qa_chain(llm, chain_type="stuff")

qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)


In [34]:
query1 = 'What are the concepts in Langchain'

In [35]:
docs = db_retriever.get_relevant_documents(query1)
docs

[Document(page_content='5/31/23, 6:13 AM Concepts — \x00\x00 LangChain 0.0.186\nhttps://python.langchain.com/en/stable/getting_started/concepts.html 1/3Concepts\nContents\nChain of Thought\nAction Plan Generation\nReAct\nSelf-ask\nPrompt Chaining\nMemetic Proxy\nSelf Consistency\nInception\nMemPrompt\nThese are concepts and terminology commonly used when developing LLM applications. It', metadata={'source': '/content/Concepts.pdf', 'page': 0}),
 Document(page_content='This is the Python specific portion of the documentation. For a purely conceptual guide to\nLangChain, see here. For the JavaScript documentation, see here.\nGetting Started\nHow to get started using LangChain to create an Language Model application.\nQuickstart Guide\nConcepts and terminology .\nConcepts and terminology', metadata={'source': '/content/WelcometoLangChain.pdf', 'page': 0}),
 Document(page_content='5/31/23, 6:10 AM Welcome to LangChain — \x00\x00 LangChain 0.0.186\nhttps://python.langchain.com/en/stable/ 2/

In [36]:
text = ''
for t in docs:
  text = text + t.page_content

In [37]:
qa_chain_route = qa_document_chain.run(input_document=text, 
                      question=query)

In [38]:
qa_chain_route

'The LangChain Agents are:\n\nChain of Thought\nAction Plan Generation\nReAct\nSelf-ask\nPrompt Chaining\nMemetic Proxy\nSelf Consistency\nInception\nMemPrompt'